# Looking at YouTube view count growth for top MVs 

## Strategy: 

1. We are going to write two scripts. One is going to get the url and title of the MV. Anotehr is going to write a script to collect the views of the music video from a specific time period. This should also contain the url. 
2. We are going to look at the top 5 trending on Youtube. 

In [1]:
%load_ext autoreload
%autoreload 2

## Importing our libraries 

In [2]:
import requests
import random
import time
from selenium.webdriver import Chrome
# from scrape_functions import (
#     get_comic_title,
#     find_image,
#     scrape_image,
#     get_prev)

In [3]:
browser = Chrome()
url = "https://www.youtube.com/feed/trending"
browser.get(url)

In [4]:
import requests

def get_mv_title(browser):
    """Return the title text"""
    title = browser.find_element_by_id('video-title')
    return title

In [5]:
def get_mv_url(title):
    """Return the url text"""
    return title.get_attribute('href')

In [6]:
from datetime import datetime

def get_view_count(browser, url):
    browser.get(url)
    time.sleep(3)
    """Return the view count"""
    today = datetime.now()
    sel = 'span.view-count'
    view_count = browser.find_element_by_css_selector(sel).text
    return int(view_count.split()[0].replace(",","")), today

In [8]:
count, today = get_view_count(browser, url)

In [ ]:
#count

In [13]:
url = "https://www.youtube.com/results?search_query=%23music+video"
browser.get(url)
time.sleep(5)
videolinks = browser.find_elements_by_id('video-title')
videos = [(link.text, link.get_attribute('href')) for link in videolinks]

info = []
for title, url in videos[:5]:
#     title = get_mv_title(video)
    view_count, today = get_view_count(browser, url)
    info.append({'title': title,
                 'url': url,
                 'view_count': view_count,
                 'today': today})

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame(info)

In [16]:
df.head()

,title,today,url,view_count
0,Cardi B & Bruno Mars - Please Me (Official Video),2019-04-26 16:04:36.969268,https://www.youtube.com/watch?v=3y-O-4IL-PU,162980871
1,City Girls - Twerk ft. Cardi B (Official Music...,2019-04-26 16:04:40.931504,https://www.youtube.com/watch?v=QryoOF5jEbc,74611782
2,"Post Malone - ""Wow."" (Official Music Video)",2019-04-26 16:04:45.497546,https://www.youtube.com/watch?v=393C3pr2ioY,49863745
3,Camila Cabello - Havana (Official Music VIdeo)...,2019-04-26 16:04:49.442050,https://www.youtube.com/watch?v=BQ0mxQXmLsk,763468761
4,Zara Larsson - Ruin My Life (Official Music Vi...,2019-04-26 16:04:53.691445,https://www.youtube.com/watch?v=3OTjFqWcDQY,41044996


## Saving the file 

In [ ]:
from pymongo import MongoClient 
from odo import odo

# open connection
connection = MongoClient()

# get db
db = connection.music_scraping.


# insert df into <table> of <dataframe>
odo(DF, db.<table>)